##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用预处理层

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/guide/keras/preprocessing_layers"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/keras/preprocessing_layers.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行 </a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/guide/keras/preprocessing_layers.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/guide/keras/preprocessing_layers.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

## Keras 预处理层

开发者可以使用 Keras 预处理层 API 来构建 Keras 原生输入处理流水线。这些输入处理流水线可在非 Keras 工作流中用作独立的预处理代码，直接与 Keras 模型组合，以及作为 Keras SavedModel 的一部分导出。

您可以使用 Keras 预处理层构建和导出真正的端到端模型：接受原始图像或原始结构化数据作为输入的模型；自行处理特征归一化或特征值索引编制的模型。

## 可用的预处理层

### 核心预处理层

- `TextVectorization` 层：将原始字符串转换为可以被 `Embedding` 层或 `Dense` 层读取的编码表示。
- `Normalization` 层：对输入特征执行逐特征归一化。

### 结构化数据预处理层

这些层用于结构化数据编码和特征工程。

- `CategoryEncoding` 层：将整数分类特征转换为独热、多热或计数密集表示。
- `Hashing` 层：执行分类特征哈希（也称为“哈希技巧”）。
- `Discretization` 层：将连续的数字特征转换为整数分类特征。
- `StringLookup` 层：将字符串分类值转换为可以被 `Embedding` 层或 `Dense` 层读取的编码表示。
- `IntegerLookup` 层：将整数分类值转换为可以被 `Embedding` 层或 `Dense` 层读取的编码表示。
- `CategoryCrossing` 层：将分类特征组合成共现特征。例如，如果您有特征值“a”和“b”，它可以提供组合特征“a 和 b 同时存在”。

### 图像预处理层

这些层用于标准化图像模型的输入。

- `Resizing` 层：将一批图像调整为目标大小。
- `Rescaling` 层：重新缩放并偏移一批图像的值（例如，从 `[0, 255]` 范围内的输入到 `[0, 1]` 范围内的输入）。
- `CenterCrop` 层：返回一批图像的中心裁剪。

### 图像数据增强层

这些层可对一批图像应用随机增强转换。它们仅在训练期间处于活动状态。

- `RandomCrop` 层
- `RandomFlip` 层
- `RandomTranslation` 层
- `RandomRotation` 层
- `RandomZoom` 层
- `RandomHeight` 层
- `RandomWidth` 层

## `adapt()` 方法

某些预处理层具有必须根据训练数据的样本计算得出的内部状态。有状态预处理层的列表如下：

- `TextVectorization`: 保留字符串标记与整数索引之间的映射
- `StringLookup` 和 `IntegerLookup`：保留输入值与输出索引之间的映射。
- `Normalization`：保留特征的平均值和标准差。
- `Discretization`：保留有关值桶边界的信息。

最关键的是，这些层**不可训练**。它们的状态并非在训练期间设置；必须在**训练之前**设置，此步骤称为“适配”。

您可以通过 `adapt()` 方法将预处理层公开给训练数据，从而设置它的状态：

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

data = np.array([[0.1, 0.2, 0.3], [0.8, 0.9, 1.0], [1.5, 1.6, 1.7],])
layer = preprocessing.Normalization()
layer.adapt(data)
normalized_data = layer(data)

print("Features mean: %.2f" % (normalized_data.numpy().mean()))
print("Features std: %.2f" % (normalized_data.numpy().std()))

`adapt()` 方法可以接受 Numpy 数组或 `tf.data.Dataset` 对象。对于 `StringLookup` 和 `TextVectorization`，您还可以传递字符串列表：

In [ ]:
data = [
    "ξεῖν᾽, ἦ τοι μὲν ὄνειροι ἀμήχανοι ἀκριτόμυθοι",
    "γίγνοντ᾽, οὐδέ τι πάντα τελείεται ἀνθρώποισι.",
    "δοιαὶ γάρ τε πύλαι ἀμενηνῶν εἰσὶν ὀνείρων:",
    "αἱ μὲν γὰρ κεράεσσι τετεύχαται, αἱ δ᾽ ἐλέφαντι:",
    "τῶν οἳ μέν κ᾽ ἔλθωσι διὰ πριστοῦ ἐλέφαντος,",
    "οἵ ῥ᾽ ἐλεφαίρονται, ἔπε᾽ ἀκράαντα φέροντες:",
    "οἱ δὲ διὰ ξεστῶν κεράων ἔλθωσι θύραζε,",
    "οἵ ῥ᾽ ἔτυμα κραίνουσι, βροτῶν ὅτε κέν τις ἴδηται.",
]
layer = preprocessing.TextVectorization()
layer.adapt(data)
vectorized_text = layer(data)
print(vectorized_text)

此外，可适配层总会公开一个选项，以直接通过构造函数参数或权重分配来设置状态。如果预期的状态值在构造层时已知，或在 `adapt()` 调用之外计算得出，则可以在不依赖层的内部计算的情况下进行设置。例如，如果 `TextVectorization`、`StringLookup` 或 `IntegerLookup` 层的外部词汇文件已经存在，则可以通过将词汇文件的路径传递到层的构造函数参数，直接将这些文件加载到查找表中。

下面是一个示例，我们使用预计算的词汇实例化一个 `StringLookup` 层：

In [ ]:
vocab = ["a", "b", "c", "d"]
data = tf.constant([["a", "c", "d"], ["d", "z", "b"]])
layer = preprocessing.StringLookup(vocabulary=vocab)
vectorized_data = layer(data)
print(vectorized_data)

## 在模型之前或模型内部预处理数据

您可以通过两种方式使用预处理层：

**选项 1**：使其成为模型的一部分，如下所示：

```python
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = rest_of_the_model(x)
model = keras.Model(inputs, outputs)
```

使用此选项，预处理将在设备端与其余模型执行同步进行，这意味着它将受益于 GPU 加速。如果您在 GPU 上进行训练，这对于 `Normalization` 层以及所有图像预处理和数据增强层来说是最佳选择。

**选项 2**：将其应用于您的 `tf.data.Dataset`，以获得生成批量预处理数据的数据集，如下所示：

```python
dataset = dataset.map(
  lambda x, y: (preprocessing_layer(x), y))
```

使用此选项，您的预处理将在 CPU 上异步进行，并在进入模型之前进行缓冲。

这对于 `TextVectorization` 和所有结构化数据预处理层来说是最佳选择。如果您在 CPU 上进行训练并使用图像预处理层，它也是一个不错的选择。

## 推断时在模型内部进行预处理的好处

即使您选择了选项 2，您之后也可能希望导出仅用于推断的端到端模型，该模型将包括预处理层。这样做的主要好处是**它使您的模型可移植**，并且**有助于降低[训练-应用偏差](https://developers.google.com/machine-learning/guides/rules-of-ml#training-serving_skew)**。

当所有数据预处理都是模型的一部分时，其他人就可以加载和使用您的模型，而无需了解如何对每个特征进行编码和归一化。您的推断模型将能够处理原始图像或原始结构化数据，而不需要模型的用户了解详细信息（例如，用于文本的分词方案，用于分类特征的索引编制方案，图像像素值被归一化为 `[-1, +1]` 还是 `[0, 1]`，等等）。如果要将模型导出到另一个运行时（如 TensorFlow.js），此功能就会尤为强大，您无需在 JavaScript 中重新实现预处理流水线。

如果您最初将预处理层放在了 `tf.data` 流水线中，则可以导出一个推断模型，将预处理打包。只需实例化一个将预处理层和训练模型链接起来的新模型：

```python
inputs = keras.Input(shape=input_shape)
x = preprocessing_layer(inputs)
outputs = training_model(x)
inference_model = keras.Model(inputs, outputs)
```

## 快速配方

### 图像数据增强（设备端）

请注意，图像数据增强层仅在训练期间处于活动状态（类似于 `Dropout` 层）。

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Create a data augmentation stage with horizontal flipping, rotations, zooms
data_augmentation = keras.Sequential(
    [
        preprocessing.RandomFlip("horizontal"),
        preprocessing.RandomRotation(0.1),
        preprocessing.RandomZoom(0.1),
    ]
)

# Create a model that includes the augmentation stage
input_shape = (32, 32, 3)
classes = 10
inputs = keras.Input(shape=input_shape)
# Augment images
x = data_augmentation(inputs)
# Rescale image values to [0, 1]
x = preprocessing.Rescaling(1.0 / 255)(x)
# Add the rest of the model
outputs = keras.applications.ResNet50(
    weights=None, input_shape=input_shape, classes=classes
)(x)
model = keras.Model(inputs, outputs)

您可以在[从头开始进行图像分类](https://keras.io/examples/vision/image_classification_from_scratch/)示例中查看类似设置的实际运作情况。

### 归一化数值特征

In [ ]:
# Load some data
(x_train, y_train), _ = keras.datasets.cifar10.load_data()
x_train = x_train.reshape((len(x_train), -1))
input_shape = x_train.shape[1:]
classes = 10

# Create a Normalization layer and set its internal state using the training data
normalizer = preprocessing.Normalization()
normalizer.adapt(x_train)

# Create a model that include the normalization layer
inputs = keras.Input(shape=input_shape)
x = normalizer(inputs)
outputs = layers.Dense(classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

# Train the model
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
model.fit(x_train, y_train)

### 通过独热编码对字符串分类特征进行编码

In [ ]:
# Define some toy data
data = tf.constant([["a"], ["b"], ["c"], ["b"], ["c"], ["a"]])

# Use StringLookup to build an index of the feature values and encode output.
lookup = preprocessing.StringLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([["a"], ["b"], ["c"], ["d"], ["e"], [""]])
encoded_data = lookup(test_data)
print(encoded_data)

请注意，索引 0 保留用于缺失值（应将其指定为空字符串`""`），索引 1 保留用于词汇外的值（未在 `adapt()` 期间看到的值）。您可以使用 `StringLookup` 的 `mask_token` 和 `oov_token` 构造函数参数进行配置。

您可以在[从头开始进行结构化数据分类](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)示例中查看 `StringLookup` 的实际运作情况。

### 通过独热编码对整数分类特征进行编码

In [ ]:
# Define some toy data
data = tf.constant([[10], [20], [20], [10], [30], [0]])

# Use IntegerLookup to build an index of the feature values and encode output.
lookup = preprocessing.IntegerLookup(output_mode="binary")
lookup.adapt(data)

# Convert new test data (which includes unknown feature values)
test_data = tf.constant([[10], [10], [20], [50], [60], [0]])
encoded_data = lookup(test_data)
print(encoded_data)

请注意，索引 0 保留用于缺失值（应将其指定为值 0），索引 1 保留用于词汇外的值（未在 `adapt()` 期间看到的值）。您可以使用 `IntegerLookup` 的 `mask_token` 和 `oov_token` 构造函数参数进行配置。

您可以在[从头开始进行结构化数据分类](https://keras.io/examples/structured_data/structured_data_classification_from_scratch/)示例中查看 `IntegerLookup` 的实际运作情况。

### 将哈希技巧应用于整数分类特征

如果您有一个可以接受许多不同值（处于 10e3 或更高的数量级）的分类特征，其中每个值只在数据中出现几次，那么对特征值进行索引和独热编码将变得不切实际且没有效果。相反，应用“哈希技巧”可能会是个好主意：将值变换成固定大小的向量。这样可以使特征空间的大小易于管理，并且无需显式索引编制。

In [ ]:
# Sample data: 10,000 random integers with values between 0 and 100,000
data = np.random.randint(0, 100000, size=(10000, 1))

# Use the Hashing layer to hash the values to the range [0, 64]
hasher = preprocessing.Hashing(num_bins=64, salt=1337)

# Use the CategoryEncoding layer to one-hot encode the hashed values
encoder = preprocessing.CategoryEncoding(num_tokens=64, output_mode="binary")
encoded_data = encoder(hasher(data))
print(encoded_data.shape)

### 将文本编码为词例索引序列

这是您应该对要传递到 `Embedding` 层的文本进行预处理的方式。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "int" output_mode
text_vectorizer = preprocessing.TextVectorization(output_mode="int")
# Index the vocabulary via `adapt()`
text_vectorizer.adapt(data)

# You can retrieve the vocabulary we indexed via get_vocabulary()
vocab = text_vectorizer.get_vocabulary()
print("Vocabulary:", vocab)

# Create an Embedding + LSTM model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = layers.Embedding(input_dim=len(vocab), output_dim=64)(x)
outputs = layers.LSTM(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

您可以在[从头进行文本分类](https://keras.io/examples/nlp/text_classification_from_scratch/)示例中查看 `TextVectorization` 层与 <code>Embedding</code> 模式组合的实际运作情况。

请注意，在训练此类模型时，为了获得最佳性能，您应将 `TextVectorization` 层用作输入流水线的一部分（我们在上面的文本分类示例中就是这样做的）。

### 使用多热编码将文本编码为 n 元语法密集矩阵

这是您应该对要传递到 `Dense` 层的文本进行预处理的方式。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "binary" output_mode (multi-hot)
# and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="binary", ngrams=2)
# Index the bigrams via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)

print("Model output:", test_output)

### 使用 TF-IDF 加权将文本编码为 n 元语法密集矩阵

这是在将文本传递到 `Dense` 层之前对文本进行预处理的另一种方式。

In [ ]:
# Define some text data to adapt the layer
data = tf.constant(
    [
        "The Brain is wider than the Sky",
        "For put them side by side",
        "The one the other will contain",
        "With ease and You beside",
    ]
)
# Instantiate TextVectorization with "tf-idf" output_mode
# (multi-hot with TF-IDF weighting) and ngrams=2 (index all bigrams)
text_vectorizer = preprocessing.TextVectorization(output_mode="tf-idf", ngrams=2)
# Index the bigrams and learn the TF-IDF weights via `adapt()`
text_vectorizer.adapt(data)

print(
    "Encoded text:\n",
    text_vectorizer(["The Brain is deeper than the sea"]).numpy(),
    "\n",
)

# Create a Dense model
inputs = keras.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

# Call the model on test data (which includes unknown tokens)
test_data = tf.constant(["The Brain is deeper than the sea"])
test_output = model(test_data)
print("Model output:", test_output)